# Make Sure You Are Ready to Go

$\renewcommand{\ket}[1]{\left| #1 \right\rangle}
\renewcommand{\bra}[1]{\left\langle #1 \right|}
\renewcommand{\braket}[2]{\left\langle #1 | #2 \right\rangle}
\newcommand{\ketbra}[2]{\left| #1 \right\rangle\!\left\langle #2 \right|}$

If you haven't done it yet, try running the following lines of code and use the [registration and installation](https://docs.classiq.io/latest/classiq_101/registration_installations/) page if you are having difficulty setting up your environment.\
Uncomment and run the following command to install or update to the latest version of the Classiq SDK (if not installed yet):

In [1]:
pip install -U classiq

In [4]:
import classiq


Uncomment and run the following command if your machine has not been

authenticated yet, you only need to run it once!

In [5]:
pip install keyrings.alt


In [6]:
#from keyrings.alt.file import PlaintextKeyring

# # Use a basic keyring backend that stores credentials insecurely (for environments like Colab or VS Code)
#keyring.set_keyring(PlaintextKeyring())

# import classiq
classiq.authenticate()

Your user code: FWPW-GRMV
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=FWPW-GRMV
Please set a password for your new keyring: ··········
Please confirm the password: ··········


Now you are good to go!

# Rydberg Phase Diagram

Before starting to code, let us reiterating some theory on Rydberg atoms - the subject of this challenge. They interact via the following Hamiltonian:

$$
H = \frac{\Omega}{2} \sum_{i=1}^N X_i
    - \delta \sum_{i=1}^N n_i
    + \sum_{i \lt j} \frac{\Omega R_b^6 }{(a|i-j|)^6} n_i n_j.
$$

You can find the phase diagram for a $51$-atom chain below. It is obtained by fixing $a=1$ and $\Omega=1$ and varying $\delta$ and $R_b$.

<img src="https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/blob/main/phase_diagram.png?raw=1" alt="Phase Diagram" width="800">


Fig.1: Phase diagram of the 1D Rydberg Hamiltonian, traced out by (left) bipartite entanglement entropy and (right) expectation value of the number of Rydberg excitations. Plots are obtained using tensor-network representation of the ground states of $H$.

In this challenge, we focus on distinguishing between the $Z2$ phase, where the ground state of $H$ has large overlap with the state $\ket{rgr\ldots gr}$, and the $Z3$ phase, where the ground state overlaps strongly with basis states of the form $\ket{\ldots rggrgg\ldots}$.

Evidently, such systems can be efficiently studied using tensor networks. However, this challenge prepares us for a more realistic scenario in which we only have access to measurement outcomes from the ground state of some Hamiltonian, and our goal is to determine which phase of matter the state belongs to.

# Loading and Processing Measurement Data

*   List item
*   List item



Training data for your model contains measurement results in randomized bases performed on a 51-qubit Rydberg atoms chain. We load training data from the .npz file in the next cell.

In [7]:
!wget https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz


--2025-05-17 16:30:46--  https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz [following]
--2025-05-17 16:30:47--  https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021888 (998K) [application/octet-stream]
Saving to: ‘training_data.npz’

training_data.npz   100%[===================>] 997.94K  --.-KB/s    in 0.04s   

2025-05-17 16:30:47 (25.0

In [8]:
import numpy as np
# You might need to make additional imports depending on your implementation

In [9]:
loaded = np.load("training_data.npz", allow_pickle=True)


unprocessed_features = loaded["features"].tolist()
unprocessed_labels = loaded["labels"].tolist()

print(f'There are {len(unprocessed_features)} data points')
print(f'There were T = {len(unprocessed_features[0])} measurements performed for each data point')
print(f'The measurements were performed on {len(unprocessed_features[0][0])} qubits')
print(f'Example: 2nd experiment result of 8th data point -> {unprocessed_features[7][1]}')
print(f'Example: label for the 8th data point -> {unprocessed_labels[7]}')

There are 20 data points
There were T = 500 measurements performed for each data point
The measurements were performed on 51 qubits
Example: 2nd experiment result of 8th data point -> ['r', '-', 'i', 'i', 'r', '+', 'r', 'g', '+', '-i', 'r', '-', '-', 'g', '-i', '-', 'r', 'g', 'i', 'i', 'i', '-', 'r', '-i', 'r', 'i', 'r', '+', 'i', 'g', '-', '-i', '-', 'g', '-i', 'i', 'r', '-', '-', '-', 'i', 'i', '-i', 'g', '-i', 'g', 'r', '-', 'r', '+', '-']
Example: label for the 8th data point -> Z2


In the above,
- $\ket{g}$ is the atomic ground state, which is a $+1$-eigenstate of Pauli $Z$
- $\ket{r}$ is the highly excited Rydberg state, which is a $-1$-eigenstate of Pauli $Z$
- $\ket{+} = \frac{1}{\sqrt2}(\ket{g} + \ket{r})$, a $+1$-eigenstate of Pauli $X$
- $\ket{-} = \frac{1}{\sqrt2}(\ket{g} - \ket{r})$, a $-1$-eigenstate of Pauli $X$
- $\ket{+i} = \frac{1}{\sqrt2}(\ket{g} +i\ket{r})$, a $+1$-eigenstate of Pauli $Y$
- $\ket{-i} = \frac{1}{\sqrt2}(\ket{g} -i \ket{r})$, a $-1$-eigenstate of Pauli $Y$.

It is up to you how to convert the features into classical shadows and labels into numbers and then both into training data for your model. For example, you could assign $-1$ to $Z2$ and $+1$ to $Z3$.

**Note:** If you decide to define any helper classes/functions in a separate Python file, please submit it alongside your solution notebook, so we can run and grade it properly

In [10]:
# ─── Cell 1: Data Loading & Classical‐Shadow Preprocessing ───
import numpy as np

# 1) Load raw measurement data
loaded = np.load("training_data.npz", allow_pickle=True)
raw_features = loaded["features"].tolist()   # 20 samples × 500 shots × 51 outcomes
raw_labels   = loaded["labels"].tolist()     # 20 labels

# 2) Build 1‐qubit shadows
g     = np.array([1,0],dtype=complex)
r     = np.array([0,1],dtype=complex)
plus  = (1/np.sqrt(2))*np.array([1,1],dtype=complex)
minus = (1/np.sqrt(2))*np.array([1,-1],dtype=complex)
p_i   = (1/np.sqrt(2))*np.array([1,1j],dtype=complex)
m_i   = (1/np.sqrt(2))*np.array([1,-1j],dtype=complex)
basis_states = {'g':g,'r':r,'+':plus,'-':minus,'+i':p_i,'-i':m_i}
I2 = np.eye(2, dtype=complex)
def sigma_op(v): return 3*np.outer(v,v.conj()) - I2
sigma_dict = {k:sigma_op(v) for k,v in basis_states.items()}

num_samples = len(raw_features)
n_qubits    = len(raw_features[0][0])
T           = len(raw_features[0])

shadows = np.zeros((num_samples,n_qubits,2,2),dtype=complex)
for i,shots in enumerate(raw_features):
    acc = np.zeros((n_qubits,2,2),dtype=complex)
    for shot in shots:
        for q,outcome in enumerate(shot):
            o = outcome.strip().replace('−','-')
            if o=='i': o = '+i'
            acc[q] += sigma_dict[o]
    shadows[i] = acc / T

# 3) Flatten, split real/imag, and truncate to first 40 angles
X_full = shadows.reshape(num_samples, -1)       # shape (20,204)
X = np.concatenate([X_full.real, X_full.imag], axis=1)  # shape (20,408)
feature_length = 40
X_sub = X[:, :feature_length]                   # shape (20,40)
num_qubits = feature_length // 2                # 20

# 4) Encode labels
label_map = {'Z2': -1, 'Z3': +1}
y = np.array([label_map[L] for L in raw_labels], dtype=int)

print("X_sub.shape:", X_sub.shape)  # (20,40)
print("y.shape   :", y.shape)      # (20,)


X_sub.shape: (20, 40)
y.shape   : (20,)


# Defining a Quantum Model

In this section, you will create a QML model for classifying the quantum phases. This will include 3 stages:
- First, you will need to decide on the data encoding scheme, i.e. loading numerical features you obtained above into the quantum circuit.
- Then, you will need to come up with an ansatz - a parametrized quantum circuit, which will be optimized to perform classification.
- Finally, to readout classical information from the quantum model, you will need to perform some sort of measurement on the resultant quantum state. Perhaps, you could extract an expectation value of some Pauli-string $P \in \{I, X, Y, Z\}^{\otimes N}$, so that $\langle P \rangle < b$ is interpreted as $Z2$ and  $\langle P \rangle > b$ is interpreted as $Z3$ for some decision boundary $b$.

There are several approaches to QML in Classiq, linked below.

You may find the following guides useful:
- QML with Classiq: http://docs.classiq.io/latest/user-guide/read/qml_with_classiq_guide/
- Variational Model Example: https://github.com/Classiq/classiq-library/blob/main/algorithms/qaoa/maxcut/qaoa_max_cut.ipynb
- Hybrid QNN: https://docs.classiq.io/latest/explore/algorithms/qml/hybrid_qnn/hybrid_qnn_for_subset_majority/

Although the 2nd guide describes a hybrid model, **you may not implement a hybrid model**, the guide should only be used as a reference as to how to implement QML.

**Warning**: Training using the Classiq PyTorch integration may take a prohibitive amount of time. Consider this when choosing an approach.

In [11]:
from classiq import *
from classiq.execution import *
import numpy as np
from classiq.synthesis import synthesize, show

In [12]:
# ─── Cell 2: Circuit Definition & Synthesis ───
from classiq import (
    qfunc, CArray, CReal, QArray, QBit, Output,
    allocate, RX, RY, RZ, CZ,
    create_model, Constraints
)
from classiq.execution import (
    ExecutionPreferences,
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames
)
from classiq.synthesis import synthesize, show

# 1) Hyperparameters
feature_length = 40
num_qubits     = 20
n_layers       = 2
num_weights    = num_qubits * 2 * n_layers

# 2) Encoding qfunc (angle‐encoding)
@qfunc
def encoding(feature: CArray[CReal, feature_length], wires: QArray) -> None:
    for q in range(num_qubits):
        θ = feature[2*q]
        ϕ = feature[2*q + 1]
        RY(theta=θ, target=wires[q])
        RZ(theta=ϕ, target=wires[q])

# 3) Ansatz qfunc (hardware‐efficient)
@qfunc
def ansatz(weights: CArray[CReal, num_weights], wires: QArray) -> None:
    ptr = 0
    for _ in range(n_layers):
        for q in range(num_qubits):
            RX(theta=weights[ptr],   target=wires[q]); ptr += 1
            RY(theta=weights[ptr],   target=wires[q]); ptr += 1
        for q in range(num_qubits - 1):
            CZ(ctrl=wires[q], target=wires[q+1])

# 4) Entry qfunc
@qfunc
def main(feature: CArray[CReal, feature_length],
         weights: CArray[CReal, num_weights],
         result:  Output[QArray[QBit]]) -> None:
    allocate(num_qubits, result)
    encoding(feature, wires=result)
    ansatz(weights,   wires=result)



Quantum program link: https://platform.classiq.io/circuit/2xENs3SRYvIUmJ2W1i0jX3uwoPO?login=True&version=0.79.1


### Synthesis

Before training, you must synthesize your model into a quantum program. Placeholders for your parameters will be automatically generated.

You may find the following documentation useful: https://docs.classiq.io/latest/sdk-reference/synthesis/

In [60]:
NUM_SHOTS = 1000
BACKEND_PREFS = ClassiqBackendPreferences(
    backend_name=ClassiqSimulatorBackendNames.SIMULATOR
)
OPT_PARAM = "no_opt"

QMOD  = create_model(
    main,
    execution_preferences=ExecutionPreferences(
        num_shots=NUM_SHOTS,
        backend_preferences=BACKEND_PREFS
    ),
    constraints=Constraints(optimization_parameter=OPT_PARAM)
)
QPROG = synthesize(QMOD)
show(QPROG)


Quantum program link: https://platform.classiq.io/circuit/2xELRD1kTCfLhhVgXfOSV8daFgm?login=True&version=0.79.1


# Training the Model

Here, you will optimize the weights in ansatz, so that the model can distiguish between the phases.

You can find the following Classiq tutorial and documentation useful:
- Execution: https://docs.classiq.io/latest/sdk-reference/execution/
- Execution Session: https://docs.classiq.io/latest/user-guide/execution/ExecutionSession/
- Executing With Parameters: https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-entry-point/

It is highly recommended to use an ExecutionSession if you are executing the same circuit with different parameters many times. It is not needed to train parameters using the Classiq PyTorch integration.

If you are not using the PyTorch integration, you will need an objective (also known as a 'loss', or 'cost') function. Depending on your implementation, you will need to either minimize or maximize it in training.

In [ ]:
# ─── Cell 3: SPSA Training Loop ───
import numpy as np
import matplotlib.pyplot as plt
from classiq.execution import ExecutionSession, ExecutionPreferences

# 1) Build sessions
def build_session(shots:int, fast:bool) -> ExecutionSession:
    prefs = ExecutionPreferences(
        num_shots=shots,
        backend_preferences=BACKEND_PREFS,
        fast=fast
    )
    return ExecutionSession(quantum_program=QPROG,
                            execution_preferences=prefs)

sess32  = build_session(32,  fast=True)
sess128 = build_session(128, fast=False)

# 2) Binder & full‐qubit Z‐avg
def _bind(prefix, arr):
    return {f"{prefix}_param_{i}": float(v) for i,v in enumerate(arr)}

def _avg_z_all(counts):
    total = sum(counts.values())
    zs = sum(
        c * sum((+1 if b=='0' else -1) for b in bits)
        for bits,c in counts.items()
    )
    return zs / (total * num_qubits)

# 3) Batched forward
def batch_forward(sess, thetas, Xb):
    plist  = [
        {**_bind("weights", th), **_bind("feature", x)}
        for th,x in zip(thetas, Xb)
    ]
    results = sess.batch_sample(parameters=plist)
    counts  = [
        r.counts if isinstance(r.counts, dict) else r.counts[0]
        for r in results
    ]
    return np.array([_avg_z_all(c) for c in counts])

# 4) Loss & accuracy
num_weights = num_qubits * 2 * n_layers
num_samples = X_sub.shape[0]

def hinge_loss(sess, w):
    logits = batch_forward(sess, np.tile(w,(num_samples,1)), X_sub)
    return np.mean(np.maximum(0,1 - y*logits))

def accuracy(sess, w):
    logits = batch_forward(sess, np.tile(w,(num_samples,1)), X_sub)
    return np.mean(np.sign(logits)==y)

# 5) SPSA hyperparams & init
alpha, c0, A, gamma = 0.15, 0.03, 10.0, 0.101
MAX_IT = 100
theta  = 0.01 * np.random.randn(num_weights)

loss_hist, acc_hist = [], []

# 6) SPSA loop
for k in range(MAX_IT):
    sess = sess32 if k<40 else sess128
    ck   = c0 / (k+1)**gamma
    ak   = alpha / (k+1+A)**0.602
    delta = np.random.choice([+1,-1], num_weights)

    lp = hinge_loss(sess, theta + ck*delta)
    lm = hinge_loss(sess, theta - ck*delta)
    grad = (lp - lm) / (2*ck*delta)
    theta -= ak * grad

    if k%10==0 or k==MAX_IT-1:
        L   = hinge_loss(sess,theta)
        Acc = accuracy(sess,theta)
        loss_hist.append(L); acc_hist.append(Acc)
        phase = "32-shot" if k<40 else "128-shot"
        print(f"iter {k:3d} | {phase:<8} | loss={L:.4f} | acc={Acc:.2%}")

# 7) Cleanup & plots
sess32.close(); sess128.close()

plt.figure(figsize=(5,2.5))
plt.plot(loss_hist,'-o'); plt.title("Hinge Loss"); plt.xlabel("Checkpoint")
plt.figure(figsize=(5,2.5))
plt.plot(acc_hist,'-o'); plt.title("Accuracy");   plt.xlabel("Checkpoint")
plt.tight_layout(); plt.show()

# 8) Final circuit
show(QPROG)


iter   0 | 32-shot  | loss=1.0109 | acc=50.00%
iter  10 | 32-shot  | loss=1.0065 | acc=50.00%
iter  20 | 32-shot  | loss=1.0064 | acc=50.00%
iter  30 | 32-shot  | loss=1.0063 | acc=50.00%
iter  40 | 128-shot | loss=1.0087 | acc=50.00%
iter  50 | 128-shot | loss=1.0055 | acc=50.00%


Training that takes too long may make it impossible to grade your submission.

# Testing the Model

Good job! Now it's time to see whether the model you designed can successfully perform the classification. For this, compare the predictions of your model to the actual labels.

If the model does not perform well, try modifying the encoding and/or the ansatz (by using different number of parameters/qubits/ansatz layers/...)

In [ ]:
### Your Code Goes Here: ###

## Grading

You will be evaluated on the accuracy, depth, width of your model and the number of parameters in your model.

The following function will return the width and depth of your model as they will be used in grading. Use it to self-evaluate your model.

In [67]:
from classiq import QuantumProgram

def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth
              - "width": Circuit width
    """
    circuit = QuantumProgram.from_qprog(qprog)

    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
    }

    return metrics

In [68]:
print(get_metrics(QPROG))

{'depth': 27, 'width': 20}


# Submission

You will submit this notebook, your trained parameters, and your quantum model.

In [ ]:
# Do not change this cell

import os

def save_qprog(qprog, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.qprog"
    with open(os.path.join(folder, file_name), 'w') as f:
        f.write(qprog.model_dump_json(indent=4))

def save_params(params, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.npz"
    with open(os.path.join(folder, file_name), 'wb') as f:
        np.savez(f, params=params)

In [ ]:
# Change to your team name!!
TEAM_NAME = ""

# Insert your trained parameters here!
TRAINED_PARAMS = []

save_qprog(QPROG, team_name=TEAM_NAME)
save_params(params=TRAINED_PARAMS, team_name=TEAM_NAME)